In [9]:
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from matplotlib import pyplot as plt
import seaborn
from pandas import DataFrame
from tqdm import tqdm
from imblearn.under_sampling import RandomUnderSampler #アンダーサンプリング用
import pickle
# 機械学習用
from sklearn.cluster import KMeans #クラスタリング用
from sklearn.ensemble import RandomForestClassifier#ランダムフォレスト
from copy import deepcopy as cp
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import AdaBoostClassifier
import lightgbm as lgb
from sklearn.linear_model import LogisticRegression as LR

pd.set_option('display.width',400)#勝手に改行コードを入れられるのを防ぐ

## 前処理&学習、本番データ切り分け

In [10]:
#学習データの読み込み
place="tokuyama"
result_df = pd.read_csv('../csv/train_{}.csv'.format(place))

result_df=result_df.drop(["Unnamed: 0"],axis=1)#csvファイルについている名無しの列を削除
#result_df=result_df.iloc[7000:]
result_df=result_df.drop(["racer_1_ID","racer_2_ID","racer_3_ID","racer_4_ID","racer_5_ID","racer_6_ID",],axis=1)#IDはいらないので削除
result_df=result_df.replace(0.0000,{"racer_1_ave_st_time":0.22})#新人のave_st_timeを0.22に
result_df=result_df.replace(0.0000,{"racer_2_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_3_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_4_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_5_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_6_ave_st_time":0.22})
result_df=result_df.replace(0.0000,{"racer_1_doub_win":0.02})#新人の着に絡む確率ave_st_timeを0.02に(新人の半期の偏差から導出)
result_df=result_df.replace(0.0000,{"racer_2_doub_win":0.02})
result_df=result_df.replace(0.0000,{"racer_3_doub_win":0.02})
result_df=result_df.replace(0.0000,{"racer_4_doub_win":0.02})
result_df=result_df.replace(0.0000,{"racer_5_doub_win":0.02})
result_df=result_df.replace(0.0000,{"racer_6_doub_win":0.02})

#レース番号ダミー化=======================================================
result_df_dummie=result_df
race_dummie_df=pd.get_dummies(result_df_dummie['number_race'])#number_raceをダミー化
for column, val in race_dummie_df.iteritems():
    result_df_dummie['race_{}'.format(int(column))]=val
result_df_dummie=result_df_dummie.drop('number_race',axis=1)

cols=list(result_df_dummie.columns)
male_cols=[s for s in cols if 'male' in s]#性別を示すカラムを取り出す

for col in male_cols:
    male_dummie_df=pd.get_dummies(result_df_dummie[col])#number_raceをダミー化
    for column, val in male_dummie_df.iteritems():
        result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

#ボート、モータもダミー化=============================================================================
cols=list(result_df_dummie.columns)
moter_cols=[s for s in cols if '_mo' in s]#モーター番号を示すカラムを取り出す
boat_cols=[s for s in cols if '_bo' in s]#ボート番号を示すカラムを取り出す
#boat もmoterも番号は1~99とする
numbers=np.arange(1, 100, 1)
empty_arr=[0]*len(result_df_dummie)
for col in moter_cols:
    for number in numbers:
        result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
    moter_dummie_df=pd.get_dummies(result_df_dummie[col])#モータ番号をダミー化
    for column, val in moter_dummie_df.iteritems():
        result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)

#boat番号をダミー化
for col in boat_cols:
    for number in numbers:
        result_df_dummie['{}_{}'.format(col,int(number))]=empty_arr
    boat_dummie_df=pd.get_dummies(result_df_dummie[col])#boat番号をダミー化
    for column, val in boat_dummie_df.iteritems():
        result_df_dummie['{}_{}'.format(col,int(column))]=val
    result_df_dummie=result_df_dummie.drop('{}'.format(col),axis=1)
result_df=result_df_dummie

#クラスタリングラベルの付与==================================================================
#クラスタリングに使わないカラムを取り除く
train_drops_df=pd.DataFrame({})
drop_col_names=['result_com','money']
train_drops_df['result_com']=result_df['result_com']
train_drops_df['money']=result_df['money']
cluster_target_df=result_df.drop(drop_col_names,axis=1)

#クラスタリング
#分けてみるクラスタの数は[8,10]の2個
#cluster_target_df　　trainのデータからリザルトと配当金を取り除いたもの
target_num_cluster=[8,10]
#test_clustaring_df=train_has_PCA_df
clustaring_df=cluster_target_df
"""
for num_cluster in target_num_cluster:
    pred = KMeans(n_clusters=num_cluster).fit_predict(cluster_target_df)
    clustaring_df['num={}'.format(num_cluster)]=pred
"""
    
#result_comと配当金を戻す
clustaring_df['result_com']=result_df['result_com']#正解ラベルを戻してあげる
clustaring_df['money']=result_df['money']#配当金を戻してあげる
df=clustaring_df



#学習、テストデータ切り分け
"""
num_data = len(df) 
num_valid = int(num_data/6.0)
valid_df = df.loc[(num_data-num_valid):num_data, :]#なるべく最新のデータを未知データに
train_df = df.loc[0:(num_data-num_valid-1), :]
train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])
"""
### 学習データのベースを作成(validとtrainに分割)
#学習、テストデータ切り分け(2019,2020のデータを検証用データに、ほかは学習)#
#df['year']=date_df['year']
df['date']=pd.to_datetime(df['date'])#文字列なのでdateを日付型に変換
df['year']=df['date'].dt.year
valid_df = df[(df['year']==2019) | ((df['year']==2020) )]#2019,2020のデータを検証用データに。
train_df =  df[(df['year']!=2019) & ((df['year']!=2020) )]#そのほかを学習データに
#学習データを切り分けたらyearはいらないから削除する
valid_df=valid_df.drop(['year'],axis=1)
train_df=train_df.drop(['year'],axis=1)
valid_df=valid_df.drop(['date'],axis=1)
train_df=train_df.drop(['date'],axis=1)

train_money=pd.Series(train_df['money'])
valid_money=pd.Series(valid_df['money'])
valid_total_df=valid_df.copy()#あとで分析用に全部のまとめ先のDf
train_total_df=train_df.copy()#あとで分析用に全部のまとめ先のDf

In [11]:
df

,date,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,...,racer_6_bo_93,racer_6_bo_94,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,result_com,money,year
0,2012-04-01,50.0,0.16,0.475,3,46.0,0.21,0.198,2,44.0,...,0,0,0,0,0,0,0,6,1900.0,2012
1,2012-04-01,35.0,0.17,0.349,3,50.0,0.22,0.374,3,39.0,...,0,0,0,0,0,0,0,84,17790.0,2012
2,2012-04-01,56.0,0.22,0.339,2,36.0,0.21,0.375,3,56.0,...,0,0,0,0,0,0,0,67,2020.0,2012
3,2012-04-01,38.0,0.17,0.508,4,48.0,0.19,0.176,2,40.0,...,0,0,0,0,0,0,0,64,6310.0,2012
4,2012-04-01,44.0,0.18,0.154,2,44.0,0.15,0.597,4,34.0,...,0,0,0,0,0,0,0,70,9490.0,2012
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19266,2020-03-31,33.0,0.14,0.416,3,35.0,0.17,0.179,2,63.0,...,0,0,0,0,0,0,0,12,730.0,2020
19267,2020-03-31,35.0,0.18,0.259,2,33.0,0.14,0.485,4,60.0,...,0,0,0,0,0,0,0,70,14660.0,2020
19268,2020-03-31,24.0,0.17,0.500,4,46.0,0.17,0.386,3,36.0,...,0,0,0,0,0,0,0,9,4170.0,2020
19269,2020-03-31,35.0,0.15,0.435,4,47.0,0.16,0.215,2,38.0,...,0,0,0,0,0,0,0,14,1480.0,2020


In [12]:
valid_df

,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,racer_3_ave_st,...,racer_6_bo_92,racer_6_bo_93,racer_6_bo_94,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,result_com,money
14797,31.0,0.17,0.387,3,27.0,0.17,0.297,2,36.0,0.19,...,0,0,0,0,0,0,0,0,20,16650.0
14798,25.0,0.18,0.442,4,27.0,0.16,0.468,4,32.0,0.16,...,0,0,0,0,0,0,0,0,42,1740.0
14799,33.0,0.18,0.448,3,26.0,0.17,0.193,2,36.0,0.16,...,0,0,0,0,0,0,0,0,3,910.0
14800,30.0,0.16,0.630,4,30.0,0.19,0.295,2,28.0,0.20,...,0,0,0,0,0,0,0,0,63,6920.0
14801,27.0,0.20,0.111,1,31.0,0.15,0.400,4,27.0,0.17,...,0,0,0,0,0,0,0,0,21,1250.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19266,33.0,0.14,0.416,3,35.0,0.17,0.179,2,63.0,0.20,...,0,0,0,0,0,0,0,0,12,730.0
19267,35.0,0.18,0.259,2,33.0,0.14,0.485,4,60.0,0.19,...,0,0,0,0,0,0,0,0,70,14660.0
19268,24.0,0.17,0.500,4,46.0,0.17,0.386,3,36.0,0.15,...,0,0,0,0,0,0,0,0,9,4170.0
19269,35.0,0.15,0.435,4,47.0,0.16,0.215,2,38.0,0.16,...,0,0,0,0,0,0,0,0,14,1480.0


In [13]:
train_df

,racer_1_age,racer_1_ave_st,racer_1_doub,racer_1_rank,racer_2_age,racer_2_ave_st,racer_2_doub,racer_2_rank,racer_3_age,racer_3_ave_st,...,racer_6_bo_92,racer_6_bo_93,racer_6_bo_94,racer_6_bo_95,racer_6_bo_96,racer_6_bo_97,racer_6_bo_98,racer_6_bo_99,result_com,money
0,50.0,0.16,0.475,3,46.0,0.21,0.198,2,44.0,0.18,...,0,0,0,0,0,0,0,0,6,1900.0
1,35.0,0.17,0.349,3,50.0,0.22,0.374,3,39.0,0.18,...,0,0,0,0,0,0,0,0,84,17790.0
2,56.0,0.22,0.339,2,36.0,0.21,0.375,3,56.0,0.22,...,0,0,0,0,0,0,0,0,67,2020.0
3,38.0,0.17,0.508,4,48.0,0.19,0.176,2,40.0,0.18,...,0,0,0,0,0,0,0,0,64,6310.0
4,44.0,0.18,0.154,2,44.0,0.15,0.597,4,34.0,0.18,...,0,0,0,0,0,0,0,0,70,9490.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14792,47.0,0.15,0.102,1,31.0,0.15,0.333,2,34.0,0.17,...,0,0,0,0,0,0,0,0,21,3270.0
14793,36.0,0.18,0.310,2,35.0,0.23,0.254,2,34.0,0.15,...,0,0,0,0,0,0,0,0,5,1130.0
14794,35.0,0.14,0.546,4,30.0,0.13,0.378,3,40.0,0.16,...,0,0,0,0,0,0,0,0,1,330.0
14795,39.0,0.16,0.414,3,50.0,0.16,0.570,4,49.0,0.16,...,0,0,0,0,0,0,0,0,5,1500.0


### target_comと、パラメータを渡してモデルを作成する関数

In [14]:
def making_model(result_com,depth,target_per):
    result_com
    print(result_com)
    
    #学習データのラベル変換==========================================================
    result_train_df=train_df.copy() 
    result_arr=[0]*len(result_train_df)
    i=0
    for result in result_train_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1
    result_train_df['result_com']=result_arr
    result_valid_df=valid_df.copy() 
    result_arr=[0]*len(result_valid_df)
    i=0
    for result in result_valid_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1

    result_valid_df['result_com']=result_arr

    result_train_df['money']=train_money
    result_valid_df['money']=valid_money
    #学習データラベル変換終わり============================================

    for_arr=np.arange(1,100,1)
    accuracy_arr=[0]*len(for_arr)
    target_per_arr=[0]*len(for_arr)
    pred_0=[0]*len(for_arr)
    gain_arr=[0]*len(for_arr)
    model_gain_arr=[0]*len(result_valid_df)
    valid_gain_arr=valid_money.values

    depths_arr=[4,5,6,7,8]
    #for depth in depths_arr:
    #    for sum_target_per in tqdm(for_arr):

    #index=sum_target_per-1
    #target_per=50+sum_target_per
    #target_per_arr[index]=target_per

    #モデルの評価指標値を格納するseries======================
    model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    model_score_s['target_com']=result_com#目標としているresult_comラベル番号
    model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
    model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
    #======================
    #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
    # 一層目の判別機のtrainデータ　:terget_result_df
    target_df=result_train_df#ベースのデータフレームをコピー
    target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
    target_1_df=target_df[target_df['result_com']==1]
    len_1=len(target_1_df)
    target_0_df=target_df[target_df['result_com']==0]
    len_0=len(target_0_df)
    target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
    target_train_df=pd.concat([target_1_df, target_0_df])
    #以下学習パート======================================================
    target_x_train=target_train_df.drop('money',axis=1)
    target_x_train=target_x_train.drop('result_com',axis=1)
    target_y_train=target_train_df['result_com']
    #テストデータ
    target_y_valid=result_valid_df['result_com']
    target_x_valid=result_valid_df.drop('money',axis=1)
    target_x_valid=target_x_valid.drop('result_com',axis=1)
    RF = RandomForestClassifier(random_state=1,n_estimators=1000,max_depth=depth)
    RF = RF.fit(target_x_train,target_y_train)


    #以下精度検証(１の正答率のみ調査)

    # 未知データに対する予測値
    predict_y_valid = RF.predict(target_x_valid)

    #[1]の正答率を見る
    pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                              , 'valid':target_y_valid})
    num_1=len(pred_valid_df[pred_valid_df['valid']==1])
    count=0
    #追加　配当金の情報も考慮する。
    gain_index=0
    model_gain_arr=[0]*len(result_valid_df)
    for _, s in pred_valid_df.iterrows():
        if ((s['pred']==1) and (s['valid']==1)):
            count+=1#的中回数
            model_gain_arr[gain_index]=valid_gain_arr[gain_index]
        gain_index+=1
    #gain_arr[index]=sum(model_gain_arr)
    #accuracy_arr[index]=(count/num_1)*100
    #try:
    #    pred_0[index]=pred_valid_df['pred'].value_counts()[0]
    #except:
    #    pred_0[index]=0
    #scoreのseriesに情報書き込み==================
    model_score_s['総収益']=sum(model_gain_arr)
    model_score_s['投資金額']=100*sum(predict_y_valid)
    model_score_s['出現数']=sum(target_y_valid)
    model_score_s['購買予測数']=sum(predict_y_valid)
    model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
    model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
    model_score_s['的中数']=count
    display(model_score_s)
    
    return RF
            

In [19]:
#test_model=making_model(result_com,depth,target_per)
target=17
depth=5
per0=129
model=making_model(target,depth,per0)
model

#モデル保存
filename = '../pickle_data/model/model_com{}_dep{}_per{}.sav'.format(target,depth,per0)

pickle.dump(model, open(filename, 'wb'))


17


target_com       17.000000
depth             5.000000
target_per      129.000000
総収益           50880.000000
投資金額          35300.000000
出現数              56.000000
購買予測数           353.000000
利益率             144.135977
購買的中率             6.515581
的中数              23.000000
dtype: float64

In [5]:
#モデル保存
#filename = '../pickle_data/model_com{}_dep{}_per{}.sav'.format(target,depth,per0)
#pickle.dump(model, open(filename, 'wb'))

### pickleデータの動作確認

In [6]:
def pickle_model(result_com,depth,target_per):
    filename = '../pickle_data/model_com{}_dep{}_per{}.sav'.format(result_com,depth,target_per)
    result_com
    print(result_com)
    
    #学習データのラベル変換==========================================================
    result_train_df=train_df.copy() 
    result_arr=[0]*len(result_train_df)
    i=0
    for result in result_train_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1
    result_train_df['result_com']=result_arr
    result_valid_df=valid_df.copy() 
    result_arr=[0]*len(result_valid_df)
    i=0
    for result in result_valid_df['result_com']:
        if ((result==result_com)):
            result_arr[i]=1
        else:
            result_arr[i]=0
        i+=1

    result_valid_df['result_com']=result_arr

    result_train_df['money']=train_money
    result_valid_df['money']=valid_money
    #学習データラベル変換終わり============================================

    for_arr=np.arange(1,100,1)
    accuracy_arr=[0]*len(for_arr)
    target_per_arr=[0]*len(for_arr)
    pred_0=[0]*len(for_arr)
    gain_arr=[0]*len(for_arr)
    model_gain_arr=[0]*len(result_valid_df)
    valid_gain_arr=valid_money.values

    depths_arr=[4,5,6,7,8]
    #for depth in depths_arr:
    #    for sum_target_per in tqdm(for_arr):

    #index=sum_target_per-1
    #target_per=50+sum_target_per
    #target_per_arr[index]=target_per

    #モデルの評価指標値を格納するseries======================
    model_score_s=pd.Series(index=['target_com','depth','target_per','総収益', '投資金額','出現数','購買予測数','利益率','購買的中率','的中数'])
    model_score_s['target_com']=result_com#目標としているresult_comラベル番号
    model_score_s['depth']=depth#ハイパーパラメータ＿木の深さ
    model_score_s['target_per']=target_per#学習データ_1に対してどの程度の0のデータを持たせるか。
    #======================
    #trainの[0]に対して、target_perの割合の量[1]を持った学習データの作成
    # 一層目の判別機のtrainデータ　:terget_result_df
    target_df=result_train_df#ベースのデータフレームをコピー
    target_df=target_df.sample(frac=1, random_state=1)#シャッフル、時系列の偏りを無くす
    target_1_df=target_df[target_df['result_com']==1]
    len_1=len(target_1_df)
    target_0_df=target_df[target_df['result_com']==0]
    len_0=len(target_0_df)
    target_0_df=target_0_df.iloc[(len_0-int(len_1*(target_per/100))):len_0]#1に対する目標の割合ぶん0の結果だったレースを抽出（後ろから抽出）
    target_train_df=pd.concat([target_1_df, target_0_df])
    #以下学習パート======================================================
    target_x_train=target_train_df.drop('money',axis=1)
    target_x_train=target_x_train.drop('result_com',axis=1)
    target_y_train=target_train_df['result_com']
    #テストデータ
    target_y_valid=result_valid_df['result_com']
    target_x_valid=result_valid_df.drop('money',axis=1)
    target_x_valid=target_x_valid.drop('result_com',axis=1)
    RF = pickle.load(open(filename, 'rb'))


    #以下精度検証(１の正答率のみ調査)

    # 未知データに対する予測値
    predict_y_valid = RF.predict(target_x_valid)

    #[1]の正答率を見る
    pred_valid_df=pd.DataFrame({'pred':predict_y_valid
                              , 'valid':target_y_valid})
    num_1=len(pred_valid_df[pred_valid_df['valid']==1])
    count=0
    #追加　配当金の情報も考慮する。
    gain_index=0
    model_gain_arr=[0]*len(result_valid_df)
    for _, s in pred_valid_df.iterrows():
        if ((s['pred']==1) and (s['valid']==1)):
            count+=1#的中回数
            model_gain_arr[gain_index]=valid_gain_arr[gain_index]
        gain_index+=1
    #gain_arr[index]=sum(model_gain_arr)
    #accuracy_arr[index]=(count/num_1)*100
    #try:
    #    pred_0[index]=pred_valid_df['pred'].value_counts()[0]
    #except:
    #    pred_0[index]=0
    #scoreのseriesに情報書き込み==================
    model_score_s['総収益']=sum(model_gain_arr)
    model_score_s['投資金額']=100*sum(predict_y_valid)
    model_score_s['出現数']=sum(target_y_valid)
    model_score_s['購買予測数']=sum(predict_y_valid)
    model_score_s['利益率']=(model_score_s['総収益']/model_score_s['投資金額'])*100
    model_score_s['購買的中率']=(count/sum(predict_y_valid))*100
    model_score_s['的中数']=count
    display(model_score_s)
    return RF
            

In [7]:
target=3
depth=6
per0=121
model=pickle_model(target,depth,per0)
model

3


target_com        3.000000
depth             6.000000
target_per      121.000000
総収益            8350.000000
投資金額          15300.000000
出現数             151.000000
購買予測数           153.000000
利益率              54.575163
購買的中率             9.150327
的中数              14.000000
dtype: float64

RandomForestClassifier(max_depth=6, n_estimators=1000, random_state=1)

In [8]:
target=4
depth=7
per0=131
model=pickle_model(target,depth,per0)
model

4


target_com        4.000000
depth             7.000000
target_per      131.000000
総収益           24060.000000
投資金額          20600.000000
出現数              88.000000
購買予測数           206.000000
利益率             116.796117
購買的中率             9.223301
的中数              19.000000
dtype: float64

RandomForestClassifier(max_depth=7, n_estimators=1000, random_state=1)

In [9]:
target=5
depth=8
per0=122
model=pickle_model(target,depth,per0)
model

5


target_com        5.000000
depth             8.000000
target_per      122.000000
総収益           16740.000000
投資金額          15700.000000
出現数             221.000000
購買予測数           157.000000
利益率             106.624204
購買的中率            12.101911
的中数              19.000000
dtype: float64

RandomForestClassifier(max_depth=8, n_estimators=1000, random_state=1)

In [10]:
target=7
depth=7
per0=146
model=pickle_model(target,depth,per0)
model

7


target_com       7.000
depth            7.000
target_per     146.000
総収益           1420.000
投資金額          3200.000
出現数            140.000
購買予測数           32.000
利益率             44.375
購買的中率            6.250
的中数              2.000
dtype: float64

RandomForestClassifier(max_depth=7, n_estimators=1000, random_state=1)

In [11]:
target=13
depth=6
per0=115
model=pickle_model(target,depth,per0)
model

13


target_com       13.000000
depth             6.000000
target_per      115.000000
総収益           61900.000000
投資金額          62900.000000
出現数              71.000000
購買予測数           629.000000
利益率              98.410175
購買的中率             4.928458
的中数              31.000000
dtype: float64

RandomForestClassifier(max_depth=6, n_estimators=1000, random_state=1)

In [12]:
target=14
depth=4
per0=123
model=pickle_model(target,depth,per0)
model

14


target_com      14.000000
depth            4.000000
target_per     123.000000
総収益           2560.000000
投資金額          7900.000000
出現数             69.000000
購買予測数           79.000000
利益率             32.405063
購買的中率            1.265823
的中数              1.000000
dtype: float64

RandomForestClassifier(max_depth=4, n_estimators=1000, random_state=1)

In [13]:
for i in target_x_valid.columns:
    print(i)

NameError: name 'target_x_valid' is not defined